In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator


data = pd.read_csv("flipkart_reviews.csv")
print(data.head())

                                        Product_name 
  0  Lenovo Ideapad Gaming 3 Ryzen 5 Hexa Core 5600...   
 1  Lenovo Ideapad Gaming 3 Ryzen 5 Hexa Core 5600...   
 1  Lenovo Ideapad Gaming 3 Ryzen 5 Hexa Core 5600...   
 2  Lenovo Ideapad Gaming 3 Ryzen 5 Hexa Core 5600...   
 3  DELL Inspiron Athlon Dual Core 3050U - (4 GB/2...   
 4  DELL Inspiron Athlon Dual Core 3050U - (4 GB/2...   

                                             Review  Rating  
 0  Best under 60k Great performanceI got it for a...       5  
 1                                 Good perfomence...       5  
 2  Great performance but usually it has also that...       5  
 3           My wife is so happy and best product 👌🏻😘       5  
 4  Light weight laptop with new amazing features,...       5  


In [ ]:
print(data.isnull().sum())

In [ ]:
import nltk
import re
nltk.download('stopwords')
stemmer=nltk.SnowballStemmer("english")
from nltk.corpus import stopwords
import string
stopword=set(stopwords.words("english"))

def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text
data["Review"] = data["Review"].apply(clean)


In [ ]:
ratings = data["Rating"].value_counts()
numbers = ratings.index
quantity = ratings.values

import plotly.express as px
figure = px.pie(data,
             values=quantity,
             names=numbers,hole = 0.5)
figure.show()

In [ ]:
text = " ".join(i for i in data.Review)
stopwords = set(STOPWORDS)
wordcloud = WordCloud(stopwords=stopwords,
                      background_color="white").generate(text)
plt.figure( figsize=(15,10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
nltk.download('vader_lexicon')
sentiments = SentimentIntensityAnalyzer()
data["Positive"] = [sentiments.polarity_scores(i)["pos"] for i in data["Review"]]
data["Negative"] = [sentiments.polarity_scores(i)["neg"] for i in data["Review"]]
data["Neutral"] = [sentiments.polarity_scores(i)["neu"] for i in data["Review"]]
data = data[["Review", "Positive", "Negative", "Neutral"]]
print(data.head())

In [ ]:
x = sum(data["Positive"])
y = sum(data["Negative"])
z = sum(data["Neutral"])

def sentiment_score(a, b, c):
    if (a>b) and (a>c):
        print("Positive 😊 ")
    elif (b>a) and (b>c):
        print("Negative 😠 ")
    else:
        print("Neutral 🙂 ")
sentiment_score(x, y, z)


In [ ]:
print("Positive: ", x)
print("Negative: ", y)
print("Neutral: ", z)


In [ ]:
import gensim.downloader as api
from sklearn.cluster import KMeans
import numpy as np

In [ ]:
wv = api.load('word2vec-google-news-300')

In [ ]:
def group_synonyms(words):
    word_vectors = [wv[word] for word in words]
    X = np.array(word_vectors)
    kmeans = KMeans(n_clusters=2, random_state=0).fit(X)
    synonyms = {}
    for i, word in enumerate(words):
        label = kmeans.labels_[i]
        if label not in synonyms:
            synonyms[label] = []
        synonyms[label].append(word)
    return synonyms

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token.isalnum() and token not in stop_words]
    return ' '.join(tokens)

In [ ]:
def generate_empathetic_response(sentiment):
    empathetic_responses = [
  {"emotion": "happy", "response": "Congratulations! You must be feeling overjoyed."},
  {"emotion": "sad", "response": "Don't worry, everything will be okay."},
  {"emotion": "neutral", "response": "I see. Well, whatever you're going through, remember that I'm here to support you."},
  {"emotion": "angry", "response": "It's understandable to feel angry. Can you tell me more about what happened?"},
  {"emotion": "frustrated", "response": "That sounds frustrating. Is there anything I can help you with?"},
  {"emotion": "excited", "response": "That's exciting news! Tell me more about it."},
  {"emotion": "scared", "response": "It's okay to feel scared. What can I do to help you feel calmer?"},
  {"emotion": "worried", "response": "Don't worry, we can figure this out together. What are you worried about?"},
  {"emotion": "grateful", "response": "That's wonderful! It's always nice to feel grateful."},
  {"emotion": "proud", "response": "You should be proud! That's a great accomplishment."},
  {"emotion": "confused", "response": "It sounds like you're feeling confused. Let me see if I can help clarify things."},
  {"emotion": "disappointed", "response": "I understand you're feeling disappointed. Is there anything I can do to help?"},
  {"emotion": "hopeful", "response": "That's a positive outlook! What are you hopeful about?"},
  {"emotion": "inspired", "response": "That's inspiring! What can I do to help you achieve your goals?"},
  {"emotion": "lonely", "response": "I understand feeling lonely.  Would you like to talk about it?"},
  {"emotion": "stressed", "response": "It sounds like you're feeling stressed.  What can I do to help you relax?"},
  {"emotion": "motivated", "response": "That's great that you're feeling motivated! What are you working towards?"},
  {"emotion": "anxious", "response": "Don't worry, take a deep breath.  Is there anything I can help you with to feel less anxious?"},
  {"emotion": "content", "response": "That sounds like a peaceful state. What makes you feel content?"},
  {"emotion": "surprised", "response": "Wow, that's surprising! Tell me more about it."},
  {"emotion": "embarrassed", "response": "Everyone feels embarrassed sometimes. Don't worry about it."},
  {"emotion": "jealous", "response": "It's okay to feel jealous sometimes. How can I help you focus on your own goals?"},
  {"emotion": "energetic", "response": "That's great that you're feeling energetic! What are you going to do with all that energy?"},
  {"emotion": "relaxed", "response": "That sounds like a nice way to be. What helped you feel relaxed?"},
  {"emotion": "curious", "response": "That's a great question!  Let's see if we can find the answer together."},
  {"emotion": "bored", "response": "I understand feeling bored.  Is there anything I can suggest to keep you entertained?"},
  {"emotion": "creative", "response": "That's fantastic! What are you feeling creative about?"},
  {"emotion": "nostalgic", "response": "It's nice to reminisce sometimes. What are you feeling nostalgic about?"},
  {"emotion": "determined", "response": "That's a great attitude to have! What are you determined to achieve?"},
  {"emotion": "sleepy", "response": "It sounds like you need some rest.  Would you like me to tell you a story?"}
]

    return empathetic_responses.get(sentiment, "I understand.")

In [ ]:
def load_stories_from_csv(file_path):
    try:
        df = pd.read_csv(file_path)
        return df['Story'].tolist()
    except FileNotFoundError:
        print("Error: CSV file not found.")
        return []

In [ ]:
def train_vectorizer(stories):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(stories)
    return vectorizer, X

In [ ]:
def get_user_input():
    return input("Enter your situation: ")

In [ ]:
def find_most_similar_story(user_input, vectorizer, X, stories):
    preprocessed_input = preprocess_text(user_input)
    user_vector = vectorizer.transform([preprocessed_input])
    similarities = cosine_similarity(user_vector, X)
    closest_story_index = similarities.argmax()
    return stories[closest_story_index]

In [ ]:
def main():
    stories = load_stories_from_csv("50_greatest_stories_of_all_time.txt")
    if not stories:
        return
    vectorizer, X = train_vectorizer(stories)
    user_input = get_user_input()
    closest_story = find_most_similar_story(user_input, vectorizer, X, stories)
    print("\nClosest story to your situation:")
    print(closest_story)

In [ ]:
if __name__ == "__main__":
    main()

Enter your situation: I just got promoted at work and I'm feeling ecstatic!
Congratulations on your promotion! You must be feeling overjoyed. To keep the positive energy flowing, I recommend reading "The Great Gatsby" by F. Scott Fitzgerald. While not directly related to your situation, this classic novel captures the essence of celebration and the pursuit of dreams, reminding us to cherish life's triumphs and enjoy the journey ahead.


In [ ]:
if __name__ == "__main__":
    main()

Enter your situation: I'm struggling to cope with the loss of a loved one.
Don't worry, everything will be okay. You should read this story about coping with loss in "The Book Thief" by Markus Zusak. The protagonist, Liesel Meminger, also experienced profound loss and grief. However, through her love for books and her connections with others, she found solace and strength to navigate through difficult times.


In [ ]:
if __name__ == "__main__":
    main()

Enter your situation: I'm feeling overwhelmed with work and responsibilities.
Don't worry, everything will be okay. You should read this story about a hardworking and determined individual in "The Shawshank Redemption" by Stephen King. Just like you, the protagonist, Andy Dufresne, faced overwhelming challenges and responsibilities. However, through perseverance and resilience, he found ways to overcome obstacles and ultimately achieve freedom and redemption.
